In [ ]:
import os
import pandas as pd
import re
from random import sample
import numpy
print(numpy.__version__)

In [ ]:
questions = pd.read_pickle('../data/ai-code/codenet-full-deepseek-1-1.code.pkl')
print(questions)

In [ ]:
code = pd.read_pickle('../data/ai-code/vl-gemini-5.code.pkl')
print(code)

In [ ]:
baseVLPath = '../datasets/VedranLjubovic/src/A2017/'

def insert_df(df: pd.DataFrame, row: list): 
  df.loc[-1] = row
  df.index = df.index + 1  # shifting index
  df = df.sort_index()  # sorting by index
  return df

def get_code_samples(path: list[str], output_data: pd.DataFrame):
  submissions = os.listdir(baseVLPath + "/".join(path) + "/")

  print(f"Problem {('/'.join(path))} has {len(submissions)} accepted C submissions")
  if(len(submissions) <= 20):
    sample_problems = submissions
  else:
    sample_problems = sample(submissions, 20)

  for problem in sample_problems:
    submission_id = re.match('student(\d+).c', problem, re.DOTALL).groups()[0]
    submission_code = open(baseVLPath + "/".join(path) + "/" + problem, 'r', encoding='utf-8').read()
    output_data = insert_df(output_data, [f"{'-'.join(path)}-{submission_id}", submission_code, 'human'])

  return output_data

def assemble_data():
  output_data = pd.DataFrame(columns=["id", "code", "actual label"])
  # problem_data = pd.DataFrame(columns=["question", "identifier"])

  for dir in os.listdir(baseVLPath):
    for dir2 in os.listdir(baseVLPath + dir + "/"):
      output_data = get_code_samples([dir, dir2], output_data)

  return output_data

In [ ]:
VL_problems = pd.read_pickle('../data/human-written/vl.code.pkl')

In [ ]:
VL_problems = assemble_data()

In [ ]:
VL_problems.to_pickle('../data/human-written/vl-2017-2.code.pkl')
print(VL_problems)

In [ ]:
print(VL_problems.loc[0].id)
print(re.match("(Z\d)-(Z\d)-\d+", VL_problems.loc[0].id, re.DOTALL).groups())